In [ ]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List, Optional
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
#import asyncio
import os
from typing import Tuple, List, Optional

import requests

from pydantic import BaseModel, ValidationInfo, model_validator

import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from src.bubble import *
from src.models import *
from src.utilities import *

In [ ]:
types_df = get("Type", constraints=[])
categories_df = get("Category")
subcategories_df = get("SubCategory")
aspects_df = get("Aspect")

In [ ]:
aspects_df.to_csv("aspects_df.csv")

In [ ]:
ID_CATEG_NONE = categories_df[categories_df["Name"].isna()].iloc[0]["_id"]
REAL_CATEGORIES = categories_df[categories_df["_id"] != ID_CATEG_NONE]
ID_SUBCATEGS_NONE = list(subcategories_df[subcategories_df["Name"].isna()]["_id"])


In [ ]:
import pandas as pd


def analyze_feedback_by_period(df, company, category, subcategory, period):
  """Analyzes customer feedback by company, category, subcategory, and period.

  Args:
      df: A pandas DataFrame containing the customer feedback data.
      company: The company to filter by.
      category: The category to filter by.
      subcategory: The subcategory to filter by.
      period: The desired period for grouping the data (e.g., "D" for day, "W" for week, "M" for month, "Y" for year).

  Returns:
      A pandas DataFrame with the following columns:
          * Date: The index of the DataFrame, containing the dates formatted according to the period, 
                  starting from the minimum date of the entire DataFrame.
          * count: The number of customer feedback entries for each period (0 if no entries).
          * ... (similar columns for rating counts, mean, and standard deviation)
          
      The min and std columns will have None values for periods with no entries.
  """

  # Get the minimum date from the entire DataFrame
  min_date = df["Date"].min()

  # Filter the DataFrame by the specified criteria

  filter = (df["Company"] == company)
  if category != ID_CATEG_NONE:
    filter &= (df["Category"] == category)
    if subcategory not in ID_SUBCATEGS_NONE:
      filter &= (df["SubCategory"] == subcategory)

  filtered_df = df[filter]

  # Handle cases where no data matches the filter
  if filtered_df.empty:
    res_df = pd.DataFrame({
        "Date": pd.date_range(start=min_date, periods=0, freq=period)})
    res_df["count"]= 0
    res_df["rating_1"]= 0
    res_df["rating_2"]= 0
    res_df["rating_3"]= 0
    res_df["rating_4"]= 0
    res_df["rating_5"]= 0
    res_df["min"]= 0
    res_df["max"]= 0
    res_df["mean"]= 0
    res_df["std"]= 0
    res_df["median"]= 0
    res_df["q1"]= 0
    res_df["q3"]= 0
    res_df["Date Display"] = res_df["Date"].apply(
      lambda x: format_date(x, period)
  )
    return res_df
  
  # Convert the "Date" column to datetime format
  filtered_df["Date"] = pd.to_datetime(filtered_df["Date"])

  # Define date format function based on period
  def format_date(date, period):
    if period == "D":
      return date.strftime("%d/%m/%Y")
    elif period == "W":
      return date.strftime("%U-%Y")  # Week format with year, e.g., 01-2024
    elif period == "M":
      return date.strftime("%m/%Y")
    else:
      return date.strftime("%Y")

  # Resample the data by the specified period, aggregating on various columns
  grouped_data = (
      filtered_df.set_index("Date")
      .resample(period)
      .agg(
          count=("Rating", "count"),
          rating_1=("Rating", lambda x: x[x == 1].count()),
          rating_2=("Rating", lambda x: x[x == 2].count()),
          rating_3=("Rating", lambda x: x[x == 3].count()),
          rating_4=("Rating", lambda x: x[x == 4].count()),
          rating_5=("Rating", lambda x: x[x == 5].count()),
          mean=("Rating", "mean"),
          std=("Rating", "std"),
          median=("Rating", "median"),
          min=("Rating", "min"),
          max=("Rating", "max"),
          q1=("Rating", lambda x: x.quantile(0.25)),
          q3=("Rating", lambda x: x.quantile(0.75)),
      )
      .reset_index()
  )

  
  # Set the minimum date from entire DataFrame as the starting index
  grouped_data.set_index("Date", inplace=True)
  # Create a date range with appropriate length based on min and max dates of filtered data
  date_range = pd.date_range(
      start=min_date, end=filtered_df["Date"].max(), freq=period
  )

  # Reindex to fill missing periods with 0 and None for mean/std
  grouped_data = grouped_data.reindex(date_range).fillna(0)

  # Fill missing values with appropriate values
  grouped_data.loc[:, "count"] = grouped_data["count"].fillna(0)
  grouped_data.loc[grouped_data["count"] == 0, ["mean", "std", "median", "min", "max", "q1", "q3"]] = None

  grouped_data = grouped_data.rename_axis('Date').reset_index()

  # Apply date formatting based on period
  grouped_data["Date Display"] = grouped_data["Date"].apply(
      lambda x: format_date(x, period)
  )
  return grouped_data


In [ ]:
def to_list(s):
    return [0 if pd.isna(x) else x for x in s]

def send_aspects(df, company, category, subcategory, period, grouped_by=None):
    d = {
        "Company": company,
        "Project": PROJECT_ID,
        "Category": category,
        "SubCategory": subcategory,
        "Period": period,
        "Grouped by": grouped_by,
        "Counts": to_list(df["count"]),
        "Counts of 1s": to_list(df["rating_1"]),
        "Counts of 2s": to_list(df["rating_2"]),
        "Counts of 3s": to_list(df["rating_3"]),
        "Counts of 4s": to_list(df["rating_4"]),
        "Counts of 5s": to_list(df["rating_5"]),
        "Dates": to_list(df["Date"].apply(lambda x:pd.to_datetime(x).strftime("%m/%d/%Y"))), #
        "Dates Display": to_list(df["Date Display"]),
        "Max Ratings": to_list(df["min"]),
        "Mean Ratings": to_list(df["mean"]),
        "Std Ratings": to_list(df["std"]),
        "Med Ratings": to_list(df["median"]),
        "Min Ratings": to_list(df["max"]),
        "Q1 Rating": to_list(df["q1"]),
        "Q3 Rating": to_list(df["q3"]),
        }  

    bubble_id = bubble_client.create("Aspect Evol",d)
    return bubble_id

In [ ]:
aspects_df

In [ ]:
subcategories_df

In [ ]:
for category in tqdm(categories_df["_id"]):
    subcategories = subcategories_df[subcategories_df["Category"]==category]["_id"]

    for subcategory in subcategories:
        for period in ["W", "Y"]: #["D","W", "M", "Y"]:
            df = analyze_feedback_by_period(aspects_df, COMPANY_ID, category, subcategory, period)
            send_aspects(df, COMPANY_ID, category, subcategory, period)

In [ ]:
df[[f"Count of {i}s" for i in range(1,6)]].plot(kind='bar', stacked=True)
